In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfTransformer

time: 322 ms (started: 2022-05-31 12:27:23 +00:00)


# Load data

## test data

In [2]:
with open("./datasets/test_leader_session.pkl" , "rb") as f:
    test_leader_dic = pickle.load(f)
    
# with open("./datasets/test_session.pkl" , "rb") as f:
#     test_dic = pickle.load(f)
    

test_leader_df = pd.DataFrame.from_dict(test_leader_dic, orient='index')
# test_df = pd.DataFrame.from_dict(test_dic, orient='index')

del test_leader_dic

time: 6.52 s (started: 2022-05-31 12:27:36 +00:00)


In [3]:
display(f"test_leader_df: {test_leader_df.shape}", test_leader_df.head())
display(test_leader_df.iloc[1,(test_leader_df.iloc[1,:]>0).tolist()], "-"*100)
display(f"test_df: {test_df.shape}", test_df.head(), "-"*100)

'test_leader_df: (50000, 904)'

,0,1,2,3,4,5,6,7,8,9,...,894,895,896,897,898,899,900,901,902,903
26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
205,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
521,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


146    4.0
176    1.0
178    3.0
183    4.0
219    2.0
242    2.0
271    1.0
290    4.0
401    4.0
440    4.0
449    1.0
452    2.0
470    1.0
489    1.0
490    3.0
507    4.0
571    4.0
663    4.0
669    4.0
692    4.0
733    1.0
743    3.0
807    4.0
855    4.0
859    4.0
873    4.0
Name: 200, dtype: float64

'----------------------------------------------------------------------------------------------------'

NameError: name 'test_df' is not defined

time: 73.8 ms (started: 2022-05-31 12:27:42 +00:00)


## all time data

In [4]:
with open("./datasets/train_session.pkl" , "rb") as f:
    train_dic = pickle.load(f)

with open("./datasets/purchases_session.pkl" , "rb") as f:
    purchases_dic = pickle.load(f)


train_df = pd.DataFrame.from_dict(train_dic, orient='index')
purchases_df = pd.DataFrame.from_dict(purchases_dic, orient='index')
del train_dic, purchases_dic

time: 6min 15s (started: 2022-05-31 12:28:19 +00:00)


In [ ]:
display(f"train_df: {train_df.shape}", train_df.head(), "-"*100)
display(f"purchases_df: {purchases_df.shape}", purchases_df.head(), "-"*100)

## 3 month data

In [ ]:
with open("./datasets/train_session_last_3m.pkl" , "rb") as f:
    train_last_3_dic = pickle.load(f)
    
with open("./datasets/purchases_session_last_3m.pkl" , "rb") as f:
    purchases_last_3_dic = pickle.load(f)
    

train_last_3_df= pd.DataFrame.from_dict(train_last_3_dic, orient='index')
purchases_last_3_df= pd.DataFrame.from_dict(purchases_last_3_dic, orient='index')
del train_last_3_dic, purchases_last_3_dic

In [ ]:
display(f"train_last_3_df: {train_last_3_df.shape}", train_last_3_df.head(), "-"*100)
display(f"purchases_last_3_df: {purchases_last_3_df.shape}", purchases_last_3_df.head(), "-"*100)

## 1 month data

In [5]:
with open("./datasets/train_session_last_1m.pkl" , "rb") as f:
    train_last_1_dic = pickle.load(f)
    
with open("./datasets/purchases_session_last_1m.pkl" , "rb") as f:
    purchases_last_1_dic = pickle.load(f)
    
with open("./datasets/train_purchases_session_last_1m.pkl" , "rb") as f:
    train_purchases_last_1_dic = pickle.load(f)
    
train_last_1_df= pd.DataFrame.from_dict(train_last_1_dic, orient='index')
purchases_last_1_df= pd.DataFrame.from_dict(purchases_last_1_dic, orient='index')
train_purchases_last_1_df= pd.DataFrame.from_dict(train_purchases_last_1_dic, orient='index')
del train_last_1_dic, purchases_last_1_dic, train_purchases_last_1_dic

time: 32.9 s (started: 2022-05-31 12:34:35 +00:00)


In [6]:
display(f"train_last_1_df: {train_last_1_df.shape}", train_last_1_df.head(), "-"*100)
display(f"purchases_last_1_df: {purchases_last_1_df.shape}", purchases_last_1_df.head(), "-"*100)
display(f"train_purchases_last_1_df: {train_purchases_last_1_df.shape}", train_purchases_last_1_df.head(), "-"*100)

'train_last_1_df: (78583, 904)'

,0,1,2,3,4,5,6,7,8,9,...,894,895,896,897,898,899,900,901,902,903
113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
115,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
325,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
332,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
388,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


'----------------------------------------------------------------------------------------------------'

'purchases_last_1_df: (78583, 904)'

,0,1,2,3,4,5,6,7,8,9,...,894,895,896,897,898,899,900,901,902,903
113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
115,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
325,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
332,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
388,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


'----------------------------------------------------------------------------------------------------'

'train_purchases_last_1_df: (78583, 904)'

,0,1,2,3,4,5,6,7,8,9,...,894,895,896,897,898,899,900,901,902,903
113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
115,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
325,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
332,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
388,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


'----------------------------------------------------------------------------------------------------'

time: 39.1 ms (started: 2022-05-31 12:35:08 +00:00)


## candidate data

In [7]:
candidate_df = pd.read_csv("./datasets/candidate_vector.csv").set_index("item_id")
candidate_ri_df = candidate_df.reset_index()

display(f"candidate: {candidate_df.shape}", candidate_df.head(), "-"*100)
display(f"candidate reset index: {candidate_ri_df.shape}", candidate_ri_df.head(), "-"*100)

'candidate: (4990, 904)'

,10_147,10_159,10_184,10_217,10_22,10_287,10_361,10_407,10_464,10_561,...,7_837,7_838,8_157,8_323,8_420,8_636,9_175,9_178,9_190,9_369
item_id,,,,,,,,,,,,,,,,,,,,,
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


'----------------------------------------------------------------------------------------------------'

'candidate reset index: (4990, 905)'

,item_id,10_147,10_159,10_184,10_217,10_22,10_287,10_361,10_407,10_464,...,7_837,7_838,8_157,8_323,8_420,8_636,9_175,9_178,9_190,9_369
0,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


'----------------------------------------------------------------------------------------------------'

time: 206 ms (started: 2022-05-31 12:35:08 +00:00)


## Concat raw data 

### concat all time data

In [ ]:
# tot_session_df =  pd.concat([test_leader_df, test_df, train_df, purchases_df])

### concat 3 month data

In [ ]:
train_con_purchasees_last_3_df = pd.concat([train_last_3_df, purchases_last_3_df])
train_last_3_con_leader_df = pd.concat([train_last_3_df, test_leader_df])

### concat 1 month data

In [8]:
train_con_purchasees_last_1_df = pd.concat([train_last_1_df, purchases_last_1_df])
train_last_1_con_leader_df = pd.concat([train_last_1_df, test_leader_df])
purchases_last_1_con_leader_df = pd.concat([purchases_last_1_df, test_leader_df])
train_purchases_last_1_con_leader_df = pd.concat([train_purchases_last_1_df, test_leader_df])

time: 342 ms (started: 2022-05-31 12:35:08 +00:00)


In [ ]:
# print(f"session_df shape:{tot_session_df.shape}")
# display(tot_session_df.head())
display(f"train_last_3_leader_df: {train_last_3_leader_df.shape}", train_last_3_leader_df.head(), "-"*100)

# Build tfidf vector

- fit tfidf model
- turn `candidte_df` to tfidf vector

In [9]:
# tfidf_train_data = train_purchases_last_1_con_leader_df
tfidf_train_data = train_last_1_con_leader_df

model = TfidfTransformer()
model.fit(tfidf_train_data)
candidate_tfidf_sparse_mat = model.transform(candidate_df)
candidate_tfidf_arr = candidate_tfidf_sparse_mat.toarray()
del candidate_tfidf_sparse_mat

time: 980 ms (started: 2022-05-31 12:35:09 +00:00)


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:438: UserWarning: X has feature names, but TfidfTransformer was fitted without feature names
  warnings.warn(


In [10]:
print(f"candidate_tfidf_arr shape:{candidate_tfidf_arr.shape}")
display(candidate_tfidf_arr[0])

candidate_tfidf_arr shape:(4990, 904)


array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

time: 5.22 ms (started: 2022-05-31 12:35:10 +00:00)


# Compute similarity 
- similarity bewteen session_tfidf and candidate_tfidf
- rank by similarity

In [11]:
def rank_tfidf_sim(target_df, overview=True):
    session_tfidf_sparse_mat = model.transform(target_df)
    session_tfidf_arr = session_tfidf_sparse_mat.toarray()
    del session_tfidf_sparse_mat
    sim_arr = np.dot(session_tfidf_arr,candidate_tfidf_arr.T) # similarity bewteen session_tfidf and candidate_tfidf
    argsort_sim_arr = np.argsort(sim_arr)[::,::-1] # rank by similarity
    item_id_rank_arr = np.apply_along_axis(lambda x:candidate_ri_df.loc[x[:100],"item_id"], 1, argsort_sim_arr).reshape(-1)
    res_df = pd.DataFrame(np.repeat(target_df.index, 100), columns=["session_id"])
    res_df["item_id"] = item_id_rank_arr
    res_df["rank"] = (list(range(1,101))*len(target_df))
    
    if (overview):
        display(f"session_tfidf_arr shape:{session_tfidf_arr.shape}",session_tfidf_arr[0], "-"*100)
        display(f"rank by similarity:", sim_arr[0][argsort_sim_arr[0]], "-"*100)
        display(f"argsort of sim_arr:", argsort_sim_arr, "-"*100)
        display(f"item_id_rank_arr[:10]:", item_id_rank_arr[:10], "-"*100)
        observe_ind = np.array([[i*100+j for j in range(10)] for i in range(len(target_df))]).reshape(-1)
        display(f"overview rank df:", res_df.iloc[observe_ind,:][:50], "-"*100)
        
    return res_df

time: 9.83 ms (started: 2022-05-31 12:35:10 +00:00)


# Output results

In [14]:
res = rank_tfidf_sim(test_leader_df)
res.to_csv("./datasets/results/leader_tfidf_train_last_1_con_leader_0531.csv", index=False)
del res

'session_tfidf_arr shape:(50000, 904)'

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.32230485, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

'----------------------------------------------------------------------------------------------------'

'rank by similarity:'

array([1.        , 0.95210322, 0.95208001, ..., 0.00663022, 0.00654687,
       0.        ])

...

'----------------------------------------------------------------------------------------------------'

'overview rank df:'

,session_id,item_id,rank
0,26,19185,1
1,26,16411,2
2,26,3260,3
3,26,17740,4
4,26,22166,5
5,26,18418,6
6,26,22736,7
7,26,7109,8
8,26,5199,9
9,26,23612,10


'----------------------------------------------------------------------------------------------------'

time: 25.9 s (started: 2022-05-31 11:25:02 +00:00)


# evaluation

In [12]:
eva_res = rank_tfidf_sim(train_df)

'session_tfidf_arr shape:(1000000, 904)'

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

'----------------------------------------------------------------------------------------------------'

'rank by similarity:'

array([0.7743349 , 0.76940127, 0.75608875, ..., 0.        , 0.        ,
       0.        ])

...

'----------------------------------------------------------------------------------------------------'

'overview rank df:'

,session_id,item_id,rank
0,3,903,1
1,3,25643,2
2,3,6048,3
3,3,22391,4
4,3,20741,5
5,3,17473,6
6,3,27603,7
7,3,3875,8
8,3,11295,9
9,3,27748,10


'----------------------------------------------------------------------------------------------------'

time: 7min 51s (started: 2022-05-31 12:35:10 +00:00)


In [13]:
display(f"eva_res shape:{eva_res.shape}", eva_res.iloc[95:105,:])

'eva_res shape:(100000000, 3)'

,session_id,item_id,rank
95,3,16958,96
96,3,2848,97
97,3,12127,98
98,3,23167,99
99,3,26253,100
100,13,27714,1
101,13,5433,2
102,13,21107,3
103,13,21215,4
104,13,2410,5


time: 3.37 ms (started: 2022-05-31 12:43:01 +00:00)


In [36]:
def mrr(evaluation_df, label_df, overview=True):
    """
    ++++++++++++++++++++++++++++++++++++
    
    `evaluation_df` example:
        | session_id | item_id | rank |
        |-----------:|--------:|-----:|
        |          3 |   16958 |   96 |
        |          3 |    2848 |   97 |
        |          3 |   12127 |   98 |
        |          3 |   23167 |   99 |
        |          3 |   26253 |  100 |
        |         13 |   27714 |    1 |
        |         13 |    5433 |    2 |
        |         13 |   21107 |    3 |
        |         13 |   21215 |    4 |
        |         13 |    2410 |    5 |
    ++++++++++++++++++++++++++++++++++++
    
    `label_df` example:
        | session_id | item_id |
        |-----------:|--------:|
        |          3 |   15085 |
        |         13 |   18626 |
        |         18 |   24911 |
        |         19 |   12534 |
        |         24 |   13226 | 
    ++++++++++++++++++++++++++++++++++++
    """
    match_filter = pd.merge(evaluation_df, label_df, on=['session_id', 'item_id'])
    rank_arr = pd.merge(label_df, match_filter, how='left', on=['session_id'], suffixes=["","_m"]).loc[:,["session_id", "item_id", "rank"]]
    rank_arr['r_rank'] = 1/rank_arr['rank']
    rank_arr = rank_arr.fillna(0)
    mrr = rank_arr['r_rank'].sum()/len(label_df)
    
    if (overview):
        display(match_filter.head(), rank_arr.head(50))
        
    return mrr

time: 500 µs (started: 2022-05-31 14:33:01 +00:00)


In [38]:
train_Y = pd.read_csv("../recsys2022/train_purchases.csv").iloc[:,:2]
mrr_score = mrr(eva_res, train_Y)
print(mrr_score)

,session_id,item_id,rank
0,31,8345,55
1,42,11784,36
2,140,23502,1
3,153,20208,17
4,154,27613,3


,session_id,item_id,rank,r_rank
0,3,15085,0.0,0.000000
1,13,18626,0.0,0.000000
2,18,24911,0.0,0.000000
3,19,12534,0.0,0.000000
4,24,13226,0.0,0.000000
5,28,26394,0.0,0.000000
6,31,8345,55.0,0.018182
7,36,14532,0.0,0.000000
8,42,11784,36.0,0.027778
9,44,4028,0.0,0.000000


0.02669856644017343
time: 16.5 s (started: 2022-05-31 14:42:45 +00:00)
